In [1]:
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import KFold

In [4]:
train = pd.read_csv('data/train_v1.csv')[train['price_doc'] <= 1e6]
test = pd.read_csv('data/test_v1.csv')
Y_0 = train['price_doc']
Y_1 = train['per_meter']
del train['price_doc']
del train['per_meter']
del test['price_doc']
del test['per_meter']

In [6]:
def rmsle(preds, Y):
        return np.sqrt(np.mean((np.log(Y+1)-np.log(preds+1))**2))
def evaluate_RF(train, test, Y):    
    '''train, test - DataFrame; Y - Series'''
    
    model = RandomForestRegressor(n_estimators=1000,n_jobs = 8)
    model_train_preds = np.zeros(train.shape[0])
    model_test_preds = np.zeros(test.shape[0])
     

    k = 0
    
    kf = KFold(n_splits = 7, random_state = 2016, shuffle = True)
    for train_index, test_index in kf.split(train, Y):
        print('Fitting fold {0}.....'.format(k))
        start = time.time()
        X_train, X_test = train.iloc[train_index], train.iloc[test_index]
        Y_train = Y.iloc[train_index]
        
        model.fit(X_train, Y_train)
        
        model_train_preds[test_index] = model.predict(X_test)
        model_test_preds += model.predict(test)
    
        k += 1
        end = time.time()
        print('Elapsed: {0}'.format(start-end))
    model_test_preds /= 7
    
    model_train_preds[model_train_preds < 0] = np.median(model_train_preds)
    model_test_preds[model_test_preds < 0] = np.median(model_test_preds)
    
    return (model_train_preds, model_test_preds, rmsle(Y, model_train_preds))

In [4]:
model_train_preds, model_test_preds, error = evaluate_RF(train,test,Y_0)

Fitting fold 0.....
Elapsed: -1220.341232061386
Fitting fold 1.....
Elapsed: -1197.131464958191
Fitting fold 2.....
Elapsed: -1226.1360960006714
Fitting fold 3.....
Elapsed: -1244.724762916565
Fitting fold 4.....
Elapsed: -1197.3302178382874
Fitting fold 5.....
Elapsed: -1182.692083120346
Fitting fold 6.....
Elapsed: -1179.2416639328003
